In [4]:
import numpy as np
import pyogrio
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from functools import reduce

insar_data_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/harmonic_egms_data_phase_filter.gpkg"
rga_risk_data_path = "C:/Users/eleonore.kong/Documents/InSAR/ARGILES/Risk_ARG.gpkg"
swi_data_path = "C:/Users/eleonore.kong/Documents/InSAR/SWI/SWI_Package_1969-2023.gpkg"

layer = 'data'
output = "C:/Users/eleonore.kong/Documents/InSAR/DATA/"

In [5]:
# sql = 'SELECT pid FROM ' + layer 
# ids = pyogrio.read_dataframe(insar_data_path, sql=sql)['pid'].tolist()

# rga_df = pyogrio.read_dataframe(rga_risk_data_path, layer='ExpoArgile_Fxx_L93')
# rga_df = rga_df.drop(['DPT','ALEA'], axis=1)

swi_sql = 'SELECT * FROM buffured_swi WHERE YEAR >= 2015 AND YEAR <= 2021'
swi_df = pyogrio.read_dataframe(swi_data_path, sql=swi_sql)

In [8]:
# swi_mean = swi_df[(swi_df['YEAR'] >= 2015) & (swi_df['YEAR'] <= 2020)]
new_data = []
swi_mean = swi_df[['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).mean().reset_index()
swi_mean = gpd.GeoDataFrame(swi_mean, crs=2154)
swi_mean = swi_mean.rename(columns={'SWI_UNIF_MENS3': 'mean_swi'})
new_data.append(swi_mean)

print("Calculate winter swi")
swi_winter = swi_df[(swi_df['MONTH'] <= 3) | (swi_df['MONTH'] >= 11)]
swi_winter = swi_winter[['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).max().reset_index()
swi_winter = gpd.GeoDataFrame(swi_winter, crs=2154)
swi_winter = swi_winter.rename(columns={'SWI_UNIF_MENS3': 'winter_max_swi'})
new_data.append(swi_winter)

print("Calculate summer swi")
swi_summer = swi_df[(swi_df['MONTH'] >= 6) & (swi_df['MONTH'] <= 9)]
swi_summer = swi_summer[['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).min().reset_index()
swi_summer= gpd.GeoDataFrame(swi_summer, crs=2154)
swi_summer = swi_summer.rename(columns={'SWI_UNIF_MENS3': 'summer_min_swi'})
new_data.append(swi_summer)

print("Calculate annual swi")
for year in [2015,2016,2017,2018,2019,2020]:
    oct_dec_swi = (swi_df['YEAR'] == year) & ((swi_df['MONTH'] >=10))
    janv_sept_swi = (swi_df['YEAR'] == year+1) & ((swi_df['MONTH'] <=9))
    data = swi_df[(oct_dec_swi) | (janv_sept_swi)][['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).mean().reset_index()
    data = data.rename(columns={'SWI_UNIF_MENS3': 'annual_mean_swi_'+str(year+1)})
    data= gpd.GeoDataFrame(data, crs=2154)
    new_data.append(data)
    
print("Calculate annual season swi")
for year in [2015,2016,2017,2018,2019,2020]:
    nov_dec = (swi_df['YEAR'] == year) & ((swi_df['MONTH'] >=11))
    janv_mars = (swi_df['YEAR'] == year+1) & ((swi_df['MONTH'] <=3))
    data = swi_df[(nov_dec) | (janv_mars)][['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).max().reset_index()
    data = data.rename(columns={'SWI_UNIF_MENS3': 'winter_max_swi_'+str(year+1)})
    data= gpd.GeoDataFrame(data, crs=2154)
    new_data.append(data)
for year in [2016,2017,2018,2019,2020,2021]:
    data = swi_df[(swi_df['YEAR'] == year) & ((swi_df['MONTH'] >=6) | (swi_df['MONTH'] <=9))][['geometry', 'SWI_UNIF_MENS3']].groupby(['geometry']).min().reset_index()
    data = data.rename(columns={'SWI_UNIF_MENS3': 'summer_min_swi_'+str(year)})
    data= gpd.GeoDataFrame(data, crs=2154)
    new_data.append(data)

print("Merging data")
swi_data = reduce(lambda left, right: pd.merge(left, right, on='geometry'), new_data)
swi_data = gpd.GeoDataFrame(swi_data, crs=2154)

swi_data['winter_summer_swi_diff'] = abs(swi_data['winter_max_swi'] - swi_data['summer_min_swi'])
for year in [2016,2017,2018,2019,2020,2021]:
    swi_data['winter_summer_swi_diff_'+str(year)] = abs(swi_data['winter_max_swi_'+str(year)] - swi_data['summer_min_swi_'+str(year)])

pyogrio.write_dataframe(swi_data, swi_data_path, layer='swi_data_agg', driver="GPKG")

Calculate winter swi
Calculate summer swi
Calculate annual swi
Calculate annual season swi
Merging data
